# Nanopore Dataset

1. Basecall and align using Dorado and minimap2:

    ```bash
    dorado basecaller \
    sup,5mCG_5hmCG \
    /path/to/pod5/ \
    --reference ref/hg38.fna \
    --trim adapters \
    --min-qscore 15 > {$SAMPLE}_sup5_unsorted.bam
    ```

2. Sort and index bam:

    ```bash
    samtools sort \
    --threads 8 \
    --write-index \
    -m 8G \
    -o "$SAMPLE.bam" \
    -O BAM \
    --reference ref/hg38.fna \
    "$SAMPLE"
    ```

3. Create a BedMethyl file with Modkit:
    
    ```bash
    modkit pileup \
    "$bam_file" \
    "$bed_file" \
    -t 32 \
    --combine-strands \
    --cpg \
    --ignore h \
    --ref ref/hg38.fna \
    --no-filtering
    ```

4. Files are ready for [ALMA-classifiers](https://pypi.org/project/alma-classifier/)

## Appendix. Create array reference BED files

In [ ]:
# import pandas as pd
# import numpy as np

# mount = '/mnt/e/ALMA/'
# output_path = mount + 'Intermediate_Files/'
# zhou2016_probes_path = mount + 'UnreliableProbesList_Zhou2016/EPIC.anno.GRCh38.tsv'

# # read df_discovery and df_validation
# df_discovery = pd.read_pickle(
#     output_path+'3314samples_331556cpgs_withbatchcorrection_bvalues.pkl').sort_index().iloc[:,1:]

# # Load the list of suboptimal probes
# array_reference = pd.read_csv(zhou2016_probes_path, sep='\t',index_col=0)

# pacmap_reference = array_reference.loc[df_discovery.columns].reset_index()

# # change values in `orientation` column to match the orientation of the probes in the discovery dataset
# pacmap_reference['orientation'] = pacmap_reference['orientation'].map({'down': '-', 'up': '+'})
# pacmap_reference['score'] = 0

# # Rename `IlmnID` to `name` to follow BED standard
# pacmap_reference = pacmap_reference.rename(columns={'IlmnID': 'name', 'orientation': 'strand'})

# # Order columns
# pacmap_reference = pacmap_reference[['chrm', 'start', 'end', 'name', 'score', 'strand', ]]

# # Sort by `chrm` and `start`
# pacmap_reference = pacmap_reference.sort_values(by=['chrm', 'start'])

# pacmap_reference.to_csv('../data/pacmap_reference.bed', sep='\t', index=False, header=False)

pacmap_reference = pd.read_csv('../data/pacmap_reference.bed', sep='\t',  names=['chrm', 'start', 'end', 'name', 'score', 'strand'])
pacmap_reference

#### OR CREATE A METHYLATION EPIC ARRAY REFERENCE BED FILE ####

# # Load the list of suboptimal probes
# array_reference = pd.read_csv(zhou2016_probes_path, sep='\t',index_col=0).reset_index()

# # rename "probeID" to "name"
# array_reference = array_reference.rename(columns={'probeID': 'name'})

# # remove rows where `chrm` is "*"
# array_reference = array_reference.loc[array_reference['chrm'] != '*'].reset_index()

# # array_reference = array_reference.loc[df_discovery.columns].reset_index()

# # change values in `orientation` column to match the orientation of the probes in the discovery dataset
# array_reference['orientation'] = array_reference['orientation'].map({'down': '-', 'up': '+'})
# array_reference['score'] = 0

# # Rename `IlmnID` to `name` to follow BED standard
# array_reference = array_reference.rename(columns={'IlmnID': 'name', 'orientation': 'strand'})

# # Order columns
# array_reference = array_reference[['chrm', 'start', 'end', 'name', 'score', 'strand', ]]

# # Sort by `chrm` and `start`
# array_reference = array_reference.sort_values(by=['chrm', 'start'])

# array_reference.to_csv('../data/array_reference.bed', sep='\t', index=False, header=False)

# array_reference = pd.read_csv('../data/array_reference.bed', sep='\t',  names=['chrm', 'start', 'end', 'name', 'score', 'strand'])
# array_reference

,chrm,start,end,name,score,strand
0,chr1,69590,69592,cg21870274,0,+
1,chr1,864702,864704,cg08258224,0,-
2,chr1,870160,870162,cg16619049,0,-
3,chr1,877158,877160,cg18147296,0,-
4,chr1,898802,898804,cg13938959,0,+
...,...,...,...,...,...,...
331551,chr9,138119084,138119086,cg00378292,0,+
331552,chr9,138120221,138120223,cg07982825,0,-
331553,chr9,138122338,138122340,cg14491707,0,+
331554,chr9,138122548,138122550,cg13811936,0,-


## Watermark

In [7]:
%load_ext watermark

In [8]:
%watermark -v -p pandas -a Francisco_Marchi@Lamba_Lab_UF -d -m -r -u

Author: Francisco_Marchi@Lamba_Lab_UF

Last updated: 2024-09-18

Python implementation: CPython
Python version       : 3.8.18
IPython version      : 8.12.3

pandas: 2.0.3

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.133.1-microsoft-standard-WSL2
Machine     : x86_64
Processor   : x86_64
CPU cores   : 32
Architecture: 64bit

Git repo: git@github.com:f-marchi/ALMA.git

